# Model Profiler API Examples

This demonstrates how to use the [profile_model](https://siliconlabs.github.io/mltk/docs/python_api/operations.html#profile-model) API.

Refer to the [Model Profiler](https://siliconlabs.github.io/mltk/docs/guides/model_profiler.html) guide for more details.

__NOTES:__  
- Click here: [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/siliconlabs/mltk/blob/master/mltk/examples/profile_model.ipynb) to run this example interactively in your browser  
- Refer to the [Notebook Examples Guide](https://siliconlabs.github.io/mltk/docs/guides/notebook_examples_guide.html) for how to run this example locally in VSCode  

## Install MLTK Python Package

In [1]:
# Install the MLTK Python package (if necessary)
!pip install --upgrade silabs-mltk

## Import Python Packages

In [1]:
# Import the standard Python packages used by the examples
import os
import urllib
import shutil
import tempfile

# Import the necessary MLTK APIs
from mltk.core import profile_model
from mltk.utils.commander import query_platform

## Download .tflite model file

A `.tflite` model file is required to run these examples.  
The following code downloads a model.

__NOTE:__ Update `TFLITE_MODEL_URL` or `tflite_path` to point to your model if necesary

In [4]:
# Use .tflite mode found here:
# https://github.com/siliconlabs/mltk/tree/master/mltk/utils/test_helper/data/
# NOTE: Update this URL to point to your model if necessary
TFLITE_MODEL_URL = 'https://github.com/siliconlabs/mltk/raw/master/mltk/utils/test_helper/data/image_example1.tflite'

# Download the .tflite file and save to the temp dir
tflite_path = os.path.normpath(f'{tempfile.gettempdir()}/image_example1.tflite')
with open(tflite_path, 'wb') as dst:
    with urllib.request.urlopen(TFLITE_MODEL_URL) as src:
        shutil.copyfileobj(src, dst)

## Example 1: Profile .tflite file in basic simulator

This example profiles the `.tflite` model file in the "basic simulator" of the model profiler.

In [5]:
# Profile the tflite model using the "basic simulator"
# NOTE: Update tflite_path to point to your model if necessary
profiling_results = profile_model(tflite_path)

# Print the profiling results
print(profiling_results)

Profiling Summary
Name: kws_ref_model
Accelerator: None
Input Shape: 1x96x96x1
Input Data Type: int8
Output Shape: 1x3
Output Data Type: int8
Model File Size (bytes): 15.7k
Runtime Memory Size (bytes): 71.3k
# Operations: 2.6M
# Multiply-Accumulates: 1.2M
# Layers: 8
# Unsupported Layers: 0
# CPU Cycles: 13.1M
CPU Utilization (%): 100.0
Clock Rate (hz): 80.0M
Time (s): 163.3m
Energy (J): 2.3m
J/Op: 884.5p
J/MAC: 2.0n
Ops/s: 16.2M
MACs/s: 7.3M
Inference/s: 6.1

Model Layers
+-------+-----------------+--------+--------+------------+------------+----------+-------------------------+--------------+-----------------------------------------------------+
| Index | OpCode          | # Ops  | # MACs | CPU Cycles | Energy (J) | Time (s) | Input Shape             | Output Shape | Options                                             |
+-------+-----------------+--------+--------+------------+------------+----------+-------------------------+--------------+-------------------------------------------

## Example 2: Profile .tflite file in MVP hardware simulator

This example profiles the `.tflite` model file in the MVP hardware accelerator simulator of the model profiler.

In [6]:
# Profile the tflite model using the MVP hardware accelerator simulator
# NOTE: Update tflite_path to point to your model if necessary
profiling_results = profile_model(tflite_path, accelerator='MVP')

# Print the profiling results
print(profiling_results)

Profiling Summary
Name: kws_ref_model
Accelerator: MVP
Input Shape: 1x96x96x1
Input Data Type: int8
Output Shape: 1x3
Output Data Type: int8
Model File Size (bytes): 15.7k
Runtime Memory Size (bytes): 71.3k
# Operations: 2.6M
# Multiply-Accumulates: 1.2M
# Layers: 8
# Unsupported Layers: 0
# Accelerator Cycles: 2.8M
# CPU Cycles: 80.4k
CPU Utilization (%): 2.8
Clock Rate (hz): 80.0M
Time (s): 35.5m
Energy (J): 315.5u
J/Op: 119.4p
J/MAC: 263.6p
Ops/s: 74.5M
MACs/s: 33.8M
Inference/s: 28.2

Model Layers
+-------+-----------------+--------+--------+------------+------------+------------+----------+-------------------------+--------------+-----------------------------------------------------+
| Index | OpCode          | # Ops  | # MACs | Acc Cycles | CPU Cycles | Energy (J) | Time (s) | Input Shape             | Output Shape | Options                                             |
+-------+-----------------+--------+--------+------------+------------+------------+----------+----------------

## Example 3: Profile .tflite file on physical device

This example profiles the `.tflite` model file on a physically connected embedded device.

__NOTE:__ A supported development board must be connected and properly enumerated for this example to work.

In [8]:
# Determine the currently connected device
# Just print an error and return if no device is connected
try:
    platform_name = query_platform()
except Exception as e:
    print(f'Failed to determine connected device, err:\n{e}')
    sys.exit(0)

print(f'Conencted device platform: {platform_name}')

accelerator = None
if platform_name in ('brd2601a', 'brd4186b'):
    # Use the MVP hardware accelerator if the platform supports it
    accelerator = 'MVP'

# Profile the tflite model on the physical device
profiling_results = profile_model(
    tflite_path,
    accelerator=accelerator,
    use_device=True
)

# Print the profiling results
print(profiling_results)

Conencted device platform: brd2601a
Profiling Summary
Name: kws_ref_model
Accelerator: MVP
Input Shape: 1x96x96x1
Input Data Type: int8
Output Shape: 1x3
Output Data Type: int8
Model File Size (bytes): 15.7k
Runtime Memory Size (bytes): 71.2k
# Operations: 2.6M
# Multiply-Accumulates: 1.2M
# Layers: 8
# Unsupported Layers: 0
# Accelerator Cycles: 2.9M
# CPU Cycles: 652.5k
CPU Utilization (%): 18.7
Clock Rate (hz): 80.0M
Time (s): 43.6m
Ops/s: 60.6M
MACs/s: 27.5M
Inference/s: 22.9

Model Layers
+-------+-----------------+--------+--------+------------+------------+----------+-------------------------+--------------+-----------------------------------------------------+
| Index | OpCode          | # Ops  | # MACs | Acc Cycles | CPU Cycles | Time (s) | Input Shape             | Output Shape | Options                                             |
+-------+-----------------+--------+--------+------------+------------+----------+-------------------------+--------------+----------------------

## Example 4: Profile model before training

Training a model can be very time-consuming, and it is useful to know how efficiently a 
model will execute on an embedded device before investing time and energy into training it.  
For this reason, the MLTK [profile_model](https://siliconlabs.github.io/mltk/docs/python_api/operations.html#profile-model) API features a `build` argument to build a model and profile it _before_ the model is fully trained.

In this example, the [image_example1](https://siliconlabs.github.io/mltk/docs/python_api/models/examples/image_example1.html) model is built
at command-execution-time and profiled in the MVP hardware simulator.  
Note that _only_ the [model specification](https://siliconlabs.github.io/mltk/docs/guides/model_specification.html)  script is required, 
it does _not_ need to be trained first.

In [9]:
# Build the image_example1 model then profile it using the MVP hardware accelerator simulator
# NOTE: Since build=True, the model does NOT need to be trained first
profiling_results = profile_model('image_example1', accelerator='MVP', build=True)

# Print the profiling results
print(profiling_results)

patool: Extracting C:/Users/reed/.mltk/downloads/rock_paper_scissors.7z ...
patool: running "C:\Program Files\7-Zip\7z.EXE" x -y -oE:/reed/mltk/tmp_archives/rock_paper_scissors -- C:/Users/reed/.mltk/downloads/rock_paper_scissors.7z
patool: ... C:/Users/reed/.mltk/downloads/rock_paper_scissors.7z extracted to `E:/reed/mltk/tmp_archives/rock_paper_scissors'.


Training:   0%|           0/3 ETA: ?s,  ?epochs/s

Epoch 1/3


0/3           ETA: ?s - 

Epoch 2/3


...

Epoch 3/3


0/3           ETA: ?s - 

INFO:tensorflow:Assets written to: E:\tmp76im2ckn\assets


c:\Users\reed\workspace\silabs\mltk\.venv\lib\site-packages\tensorflow\lite\python\convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


Profiling Summary
Name: my_model
Accelerator: MVP
Input Shape: 1x96x96x1
Input Data Type: int8
Output Shape: 1x3
Output Data Type: int8
Model File Size (bytes): 14.7k
Runtime Memory Size (bytes): 71.3k
# Operations: 2.6M
# Multiply-Accumulates: 1.2M
# Layers: 8
# Unsupported Layers: 0
# Accelerator Cycles: 2.8M
# CPU Cycles: 80.4k
CPU Utilization (%): 2.8
Clock Rate (hz): 80.0M
Time (s): 35.5m
Energy (J): 315.5u
J/Op: 119.4p
J/MAC: 263.6p
Ops/s: 74.5M
MACs/s: 33.8M
Inference/s: 28.2

Model Layers
+-------+-----------------+--------+--------+------------+------------+------------+----------+-------------------------+--------------+-----------------------------------------------------+
| Index | OpCode          | # Ops  | # MACs | Acc Cycles | CPU Cycles | Energy (J) | Time (s) | Input Shape             | Output Shape | Options                                             |
+-------+-----------------+--------+--------+------------+------------+------------+----------+---------------------